testing out geoRf
https://link.springer.com/epdf/10.1007/s10618-024-01046-7?sharing_token=DB2gHHSrA9k0q5VNIJVBG_e4RwlQNchNByi7wbcMAY4XLRxsFrveO-rybNuyvriMu7E-XYilawHRGIJBQH8NkTj8hHa1vSE8h21nCTiJBu4eL4O0jBs4XshNJ-0PXJm2LLAyc4JUqPvBbx3RPLOAcn2nYHy3Y-JXhlIYonmPB0Y%3D 

In [ ]:
#!git clone https://github.com/margotgeerts/geoRF.git


In [1]:
#!pip install -r geoRF/requirements.txt


In [2]:
import io
import requests
from urllib import request
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split



In [3]:
train_set_new = pd.read_csv('train_set_new.csv')
test_set = pd.read_csv('test_set.csv')


In [4]:
train_set_new.head()



,id,mean_gs,sd,skewness,kurtosis,current_max,current_mean,current_min,gebco,x,...,sample_type,y_im,x_im,x_m,y_m,slope,aspect,rough,bpi,current_range
0,629,4.49,2.61,0.82,2.47,0.103564,0.078791,0.058108,-195,-58.95333,...,Grab Van Veen,81,1189,242552.354220,37570.328143,1.537367,43.063620,30,0,0.045456
1,2498,2.32,0.97,4.91,37.04,0.135892,0.095632,0.069390,-2,-63.48517,...,Grab Unknown,477,101,20603.690308,221247.487952,88.378340,90.003976,-32766,8190,0.066502
2,483,2.96,0.98,4.22,21.80,0.139822,0.102822,0.081560,-22,-63.38500,...,Grab Van Veen,477,125,25499.616718,221247.487952,0.339245,294.453350,6,0,0.058263
3,598,7.45,2.49,-0.99,2.79,1.177483,1.115310,1.021761,-1965,-61.66700,...,Piston Benthos,945,537,109546.353420,438320.494999,2.803218,318.664589,47,3,0.155721
4,222,1.80,0.75,4.52,40.95,0.131379,0.122668,0.118677,-22,-59.75257,...,Grab Van Veen,635,997,203384.942941,294532.819391,0.460235,336.259696,7,0,0.012701


In [5]:

# Convert X-Y coordinates to a projected coordinate system
gdf = gpd.GeoDataFrame(train_set_new, geometry=gpd.points_from_xy(train_set_new.x, train_set_new.y, crs='EPSG:4326'))

gdf = gdf.to_crs('EPSG:3857')

train_set_new['x'] = gdf.geometry.x
train_set_new['y'] = gdf.geometry.y

gdf = gpd.GeoDataFrame(test_set, geometry=gpd.points_from_xy(test_set.x, test_set.y, crs='EPSG:4326'))

gdf = gdf.to_crs('EPSG:3857')
test_set['x'] = gdf.geometry.x
test_set['y'] = gdf.geometry.y
#renameing for easyness 
data_train = train_set_new
data_test = test_set

#data_train['mean_real'] = -2**data_train['mean_gs']
#data_test['mean_real'] = -2**data_test['mean_gs']

# Split data into train and test
#data_train, data_test = train_test_split(df, test_size=0.3, shuffle=True, random_state=0)

# Scale the target value
scaler = MinMaxScaler()
data_train['mean_gs'] = scaler.fit_transform(data_train['mean_gs'].values.reshape(-1,1))
data_test['mean_gs'] = scaler.transform(data_test['mean_gs'].values.reshape(-1,1))

In [6]:
target = ['mean_gs']
xtra_drop = ['id', 'sd', 'skewness', 'kurtosis', 'sample_type', 'y_m', 'x_m' ]
X_train = data_train.drop(target + xtra_drop, axis=1).values
y_train = data_train[target].values

X_test = data_test.drop(target + xtra_drop, axis=1).values

y_test = data_test[target].values

In [7]:
data_train.drop(target + xtra_drop, axis=1).head

<bound method NDFrame.head of       current_max  current_mean  current_min  gebco             x  \
0        0.103564      0.078791     0.058108   -195 -6.562655e+06   
1        0.135892      0.095632     0.069390     -2 -7.067137e+06   
2        0.139822      0.102822     0.081560    -22 -7.055986e+06   
3        1.177483      1.115310     1.021761  -1965 -6.864739e+06   
4        0.131379      0.122668     0.118677    -22 -6.651626e+06   
...           ...           ...          ...    ...           ...   
1683     0.126147      0.108262     0.074931   -189 -6.576199e+06   
1684     0.556092      0.522299     0.489559  -1379 -6.667566e+06   
1685     0.131255      0.105794     0.078105   -194 -6.711638e+06   
1686     0.109486      0.084734     0.056825   -205 -6.661544e+06   
1687     0.185622      0.171560     0.157634   -129 -6.691044e+06   

                 y  y_im  x_im      slope      aspect  rough   bpi  \
0     5.821039e+06    81  1189   1.537367   43.063620     30     0   
1

In [8]:
X_train[:,[4,5]]

array([[-6562654.67603713,  5821038.88379365],
       [-7067136.79722999,  5559288.59454462],
       [-7055985.92387976,  5559263.57836902],
       ...,
       [-6711638.00342804,  5564502.82816824],
       [-6661544.23235646,  5608412.40906916],
       [-6691043.89749106,  5672832.02301644]])

In [ ]:
from geoRF.geoRF import GeoRFRegressor
georf_temp = GeoRFRegressor(n_estimators=1000, max_features=None, n_jobs=-1, random_state=42)
georf_temp.fit(X_train,
               y_train,
               geo_features=[4,5], # X-Y column indices
               gens='da')  # Dual Annealing (DA) geospatial split generator
     

 14%|█▍        | 140/1000 [43:46<5:03:45, 21.19s/it]

In [ ]:




from sklearn.metrics import r2_score
# Make predictions
y_train_pred = georf_temp.predict(X_train)
#print(y_pred)
# Calculate the accuracy
accuracy_train = r2_score(y_train, y_train_pred)
print(f"r2_score on trained data: {accuracy_train}")

y_test_pred = georf_temp.predict(X_test)
#print(y_pred)
# Calculate the accuracy



accuracy_test = r2_score(y_test, y_test_pred)
print(f"r2_score on test data: {accuracy_test}")



In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

model =  ExtraTreesRegressor(n_estimators=10000, random_state=42)


# Train the model
model.fit(X_train, y_train.ravel())

In [ ]:
y_test_pred_extra = model.predict(X_test)
#print(y_pred)
# Calculate the accuracy



accuracy_test = r2_score(y_test, y_test_pred_extra)
print(f"r2_score on test data: {accuracy_test}")
